In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import pathlib
import os

from helper_functions import *
metadata = pd.read_csv("metadata.csv")
metadata_cfdna = metadata[metadata["Sample type"].isin(["cfDNA", "Control"])]
import warnings
warnings.filterwarnings("ignore")
import random

for topK in [500, 1000]:
    for atlas_sample_types in ["Tissue", "Tissue,WBC", "Tissue,WBC,Control", "Tissue,Control"]:
        datadir = "./datadir"
        outputdir = "./outputdir_02102023"
        
        path_to_03_output = os.path.join(outputdir, "03_output_noFDR")
        path_to_04_output = os.path.join(outputdir, "04_output_noFDR", "top{}_{}".format(topK, atlas_sample_types.replace(",", "_and_")))
        path_to_05_output = os.path.join(outputdir, "05_output_noFDR", "top{}_{}".format(topK, atlas_sample_types.replace(",", "_and_")))
        path_to_06_output = os.path.join(outputdir, "06_output_noFDR", "top{}_{}".format(topK, atlas_sample_types.replace(",", "_and_")))
        path_to_07_output = os.path.join(outputdir, "07_output_noFDR", "top{}_{}".format(topK, atlas_sample_types.replace(",", "_and_")))
        
        os.system("mkdir -p {}".format(path_to_07_output))
        
        selected_metadata = metadata[metadata["Sample type"].isin(["Control"])]
        
        atlas = pd.read_csv(os.path.join(path_to_03_output, "top{}_atlas_{}.final.csv".format(topK, atlas_sample_types)), index_col =[0])
        atlas = atlas[[item for item in atlas.columns if "_y" not in item ]]
        atlas.columns = [item.replace("_x", "") for item in atlas.columns]
        avg_atlas = atlas.set_index("sample").fillna(0).groupby("label").mean()
        atlas_regions = [item for item in atlas.columns if item not in ["sample", "label"]]
        
        
        for sampleid in metadata[metadata["Sample type"] == "Control"]["filename"].unique():
            for spike_in_type in ["Liver", "Gastric", "Lung", "Breast", "CRC"]:
                for spike_in_rate in [0.01, 0.05, 0.1, 0.15, 0.20, 0.30]:
                    path_to_save_spikein = os.path.join(path_to_07_output, sampleid, "spike_in_rate_{}".format(spike_in_rate),
                                                       spike_in_type)
                    
                    os.system("mkdir -p {}".format(path_to_save_spikein))
                    
                    tissue_samples = metadata[(metadata["Label"] == spike_in_type) & (metadata["Sample type"] == "Tissue")]["filename"].unique()
                    sampledf = pd.read_csv(os.path.join(path_to_06_output, "sample_readdf", "{}.readdf.csv".format(sampleid)), index_col = [0])
                    num_reads = sampledf.shape[0]
                    
                    tissue_readdf = pd.DataFrame()
                    
                    for tissue_sampleid in tissue_samples:
                        path_to_file = os.path.join(path_to_06_output, "sample_readdf", "{}.readdf.csv".format(tissue_sampleid))
                        tissue_readdf = pd.concat([tissue_readdf, pd.read_csv(path_to_file, index_col = [0])], axis = 0)
                    
                    for count in range(1, 11):
                        sampling_tissuedf = tissue_readdf.sample(int(spike_in_rate * num_reads))
                        sampling_tissuedf["sample"] = sampleid
                        spike_in_sampledf = pd.concat([sampledf, sampling_tissuedf], axis = 0)
                        spike_in_sampledf.to_csv(os.path.join(path_to_save_spikein, "{}_{}.csv".format(sampleid, count)))